In [1]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from matplotlib import pyplot as plt
from torch.autograd import Variable
from mpl_toolkits.mplot3d import Axes3D
import sklearn.metrics as metrics
import pickle
import time
from sklearn.kernel_approximation import RBFSampler
%matplotlib notebook
from utils import *

In [2]:
findex = 1
xtmp,y = get_features(findex,cleaned=True)
xtmp,y = shuffle(xtmp,y)
y = y.astype(int)
no_ex = xtmp.shape[0]

### X_features are real valued features. We apply kernel on it 

In [3]:
feat_dim = 250

rbf_feature = RBFSampler(gamma=1,n_components = feat_dim, random_state=1)
x = rbf_feature.fit_transform(xtmp)

In [ ]:
class c_mlp(nn.Module):
    def __init__(self):
        super(c_mlp,self).__init__()
        self.l1 = nn.Linear(feat_dim,1000)
        self.l2 = nn.Linear(1000,1000)
        self.l3 = nn.Linear(1000,500)
        self.l4 = nn.Linear(500,500)
        self.l5 = nn.Linear(500,500)
        self.l6 = nn.Linear(500,500)
        self.l6 = nn.Linear(500,50)
        self.l7 = nn.Linear(50,2)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        x = F.leaky_relu(self.l3(x))
        x = F.leaky_relu(self.l4(x))
        x = F.leaky_relu(self.l5(x))
        x = F.leaky_relu(self.l6(x))
        x = self.l7(x)
        
        return x

In [ ]:
max_percent = 0.5

for i in range(1,6):
    val_iter = i
    print("val iter: ",val_iter)
    
   
    #weights_array = [7]
    weights_array = np.linspace(200,300,10)
    for i,w in enumerate(weights_array): 
        mymlp = c_mlp().cuda()
        optimizer = torch.optim.Adagrad(mymlp.parameters(),lr=1e-3)
        criterion = nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor([1,w]))

        for ep in range(3000):
            
            ind = get_train_ind(val_iter=val_iter,no_examples=no_ex)
            xtrain,ytrain = get_train_batch(x,y,batch_size=90,indices=ind)
            
            model_op = mymlp(xtrain)
           
            loss = criterion(model_op,ytrain)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    #     if(ep%30==29):
    #         print(loss.data[0])

        ## After training check on cross validation data
        xval,yval = get_val_data(x,y,no_examples=no_ex,val_iter=val_iter)
        yval = yval.reshape(yval.shape[0],)
        train_op = mymlp(xval)
        train_op = train_op.cpu().data.numpy()
        pred_labels = np.argmax(train_op,axis=1)
        min_fp = xval.size()[0]*0.2
        cf = metrics.confusion_matrix(yval,pred_labels).ravel()
        [tn,fp,fn,tp] = cf
        print('tn, fp, fn, tp: ',cf)
        percent = float(float(tp)/float(tp+fn))
        if(percent>max_percent):
            if(1==1):
                if(fp < min_fp):
                    min_fp = fp
                    max_percent = percent
                    print("min fp, max_percent",fp,percent)
                    model_path = os.getcwd() + '/kernel_clean' + str(findex)
                    torch.save(mymlp.state_dict(),model_path)
                    print("saving model on val: ",val_iter," and weight: ",w)

        


('val iter: ', 1)
('tn, fp, fn, tp: ', array([8246, 1046,  137,   47]))
('tn, fp, fn, tp: ', array([8268, 1024,  140,   44]))
('tn, fp, fn, tp: ', array([8494,  798,  136,   48]))
('tn, fp, fn, tp: ', array([7103, 2189,  111,   73]))
('tn, fp, fn, tp: ', array([8002, 1290,  129,   55]))
('tn, fp, fn, tp: ', array([7444, 1848,  122,   62]))
('tn, fp, fn, tp: ', array([8197, 1095,  131,   53]))
('tn, fp, fn, tp: ', array([8374,  918,  136,   48]))
('tn, fp, fn, tp: ', array([8201, 1091,  130,   54]))
('tn, fp, fn, tp: ', array([7298, 1994,  118,   66]))
('val iter: ', 2)
('tn, fp, fn, tp: ', array([7743, 1549,  117,   68]))
('tn, fp, fn, tp: ', array([8016, 1276,  127,   58]))
('tn, fp, fn, tp: ', array([8119, 1173,  132,   53]))
('tn, fp, fn, tp: ', array([8477,  815,  143,   42]))
('tn, fp, fn, tp: ', array([7287, 2005,  111,   74]))
('tn, fp, fn, tp: ', array([8195, 1097,  127,   58]))
('tn, fp, fn, tp: ', array([8132, 1160,  135,   50]))
('tn, fp, fn, tp: ', array([7515, 1777,  113, 

### Test the model 

In [ ]:
feat_dim = 250

rbf_feature = RBFSampler(gamma=1,n_components = feat_dim, random_state=1)
trans = rbf_feature.fit(X_features)
c_fingerprints_test = trans.transform(X_features_test)

In [ ]:
testmlp = c_mlp().cuda()
model_path = os.getcwd() + '/kernel_mac' + fname + '_3'
testmlp.load_state_dict(torch.load(model_path))

In [ ]:
op = testmlp(Variable(torch.cuda.FloatTensor(c_fingerprints_test)))
op = op.cpu().data.numpy()
pred_labels = np.argmax(op,axis=1)
cf = metrics.confusion_matrix(labels2_t,pred_labels).ravel()
#print(val_iter,w)
print('tn, fp, fn, tp: ',cf)

In [ ]:
np.sum(labels2_t)